In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1 General Port Parameters

In [2]:
class general_parameters():
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.year              = 2018   # current year within simulation
        self.simulation_window = 20     # looking 20 years ahead
        self.start_year        = 2018   # start year of simulation
        self.timestep          = self.year - self.start_year
        self.operational_hours = 8760   # operational hours per year
       
parameters = general_parameters()
timestep   = parameters.timestep

# 2 Scenario Generator

In [3]:
from terminal_optimization.forecast_package import maize, soybean, wheat, handysize, handymax, panamax
from terminal_optimization.plot_package import plot_trend

In [4]:
# Generating a single demand forecast

trend_type     = 'linear'   # linear, constant or random

demand_maize   = 1000000    # demand at t=0
growth_maize   = 100000     # year on year absolute growth of demand        - input for linear method
rate_maize     = 1.01       # year on year growth rate of demand (% points) - input for constant method and random method
mu_maize       = 0          # avg bonus rate added to base rate (% points)  - input for random method
sigma_maize    = 0          # standard deviation of bonus rate (% points)   - input for random method

demand_soybean = 0
growth_soybean = 0
rate_soybean   = 0
mu_soybean     = 0
sigma_soybean  = 0

demand_wheat   = 0
growth_wheat   = 0
rate_wheat     = 0
mu_wheat       = 0
sigma_wheat    = 0

maize.generate_forecast  (trend_type, parameters.year, parameters.simulation_window, demand_maize,   growth_maize  , rate_maize,   mu_maize,   sigma_maize)
soybean.generate_forecast(trend_type, parameters.year, parameters.simulation_window, demand_soybean, growth_soybean, rate_soybean, mu_soybean, sigma_soybean)
wheat.generate_forecast  (trend_type, parameters.year, parameters.simulation_window, demand_wheat,   growth_wheat  , rate_wheat,   mu_wheat,   sigma_wheat)

# 3 Port Model
## 3.1 Import/export of data
*The following packages are imported:* 
- [infrastructure package]: Imports all infrastructure classes 
- [investment decision package]: Combines current commodity demand and infrastructure characteristics to decide whether to invest in terminal elements
<br>

*The following overacrching classes are exported:* 
- port parameters (e.g. simulation timestep, operational hours etc.)
- commodities (includes forecast characteristics)
- vessels 

In [5]:
import terminal_optimization.investment_decision_package as invest
import terminal_optimization.infrastructure_package      as infra
import terminal_optimization.business_logic_package      as financial

def export_to_package(package):
    package.parameters = parameters
    package.handysize  = handysize
    package.handymax   = handymax
    package.panamax    = panamax
    package.maize      = maize
    package.soybean    = soybean
    package.wheat      = wheat
    package.import_notebook_parameters()
    
export_to_package(invest)
export_to_package(infra)
export_to_package(financial)

## 3.2 Investment decisions

### 3.2.1 Berth investment decision
Starting with a single berth and asuming that vessels are distributed equally between all berths, the berth occupancy is calculated. If the occupancy is above the set 'allowable berth occupancy' an extra berth is added and the calculation is iterated

In [6]:
# at t=0 import berth class from infrastructure package and run initial berth configuration
if parameters.timestep == 0:
    berths = infra.berths
    berths = invest.initial_berth_setup(berths)
    
# for each time step, check whether pending assets come online
berths = invest.berth_online_transition(berths)
    
# for each time step, decide whether to invest in berths
berth_invest_decision = invest.berth_invest_decision(berths)

# if investments are needed, calculate how much berths should be added
if berth_invest_decision == 'Invest in berths':
    berths = invest.berth_expansion(berths)
else:
    berths[0].delta = 0

print ('Number of berths added:  ', berths[0].delta)
print ('Pending number of berths:', berths[0].pending_quantity)
print ('Current number of berths:', berths[0].online_quantity)

Number of berths added:   1
Pending number of berths: 1
Current number of berths: 0


### 3.2.2 Quay investment decision
In this setup, the decision to expand the quay is solely a result of the *Berth investment decision*. The length of the quay is calculated as the sum of the length of the berths 

In [7]:
# at t=0 import quay class from infrastructure package and run initial quay configuration
if parameters.timestep == 0:
    quays = infra.quays
    quays = invest.initial_quay_setup(quays)

# for each time step, check whether pending assets come online
quays = invest.quay_online_transition(quays)
    
# for each time step, decide whether to invest in the quay
quay_invest_decision = invest.quay_invest_decision(berths, quays)

# if investments are needed, calculate how much quay length should be added
if quay_invest_decision == 'Invest in quay':
    quays = invest.quay_expansion(quays, berths)
else:
    quays[0].delta = 0
    
print ('Meters of quay added: ', quays[0].delta)
print ('Current quay length:  ', quays[0].online_length)

Meters of quay added:  195
Current quay length:   0


### 3.2.3 Crane investment decision
In this setup, the number of cranes is solely goverened by the number of berths. The number of cranes per berth is equal to the number of cranes that can work simultaeously on the largest vessel that calls to port during the current timestep. E.g. two cranes per berth in years where handymax is the largest vessel and three cranes per berth in years where panamax is the largest vessel

In [8]:
# at t=0 import cranes class from infrastructure package and run initial crane configuration
if parameters.timestep == 0:
    cranes = infra.cranes
    cranes = invest.initial_crane_setup(cranes)
    
# for each time step, check whether pending assets come online
cranes = invest.crane_online_transition(cranes)
    
# for each time step, decide whether to invest in the cranes
crane_invest_decision = invest.crane_invest_decision(cranes, berths)

# if investments are needed, calculate how much cranes should be added
if crane_invest_decision == 'Invest in cranes':
    cranes = invest.crane_expansion(cranes, berths)
else:
    for i in range (4):
        cranes[i][0].delta = 0

print ('Number of berths added:', berths[0].delta)
print ('Gantry cranes added:   ', cranes[0][0].delta)
print ('Harbour cranes added:  ', cranes[1][0].delta)
print ('Mobile cranes added:   ', cranes[2][0].delta)
print ('Screw unloaders added: ', cranes[3][0].delta)

Number of berths added: 1
Gantry cranes added:    0
Harbour cranes added:   0
Mobile cranes added:    2
Screw unloaders added:  0


### 3.2.3 Storage investment decision
In this setup, the storage investment is triggered whenever the storage capacity equals 10% of yearly demand. Once triggered, the storage is expanded to accomodate 20% of yearly throughput

In [9]:
# at t=0 import storage class from infrastructure package and run initial storage configuration
if parameters.timestep == 0:
    storage = infra.storage
    storage = invest.initial_storage_setup(storage)
    
# for each time step, check whether pending assets come online
storage = invest.storage_online_transition(storage)
    
# for each time step, decide whether to invest in storage
invest_decision = invest.storage_invest_decision(storage)
storage_invest_decision = invest_decision[0]
storage = invest_decision[1]

# if investments are needed, calculate how much extra capacity should be added
if storage_invest_decision == 'Invest in storage':
    storage = invest.storage_expansion(storage)
else:
    storage[0][0].delta = 0
    storage[1][0].delta = 0

print ('Silo capacity added (t):       ', storage[0][0].delta)
print ('Current silo capacity (t):     ', storage[0][0].online_capacity)
print ('Warehouse capacity added (t):  ', storage[1][0].delta)
print ('Current warehouse capacity (t):', storage[1][0].online_capacity)

Silo capacity added (t):        0
Current silo capacity (t):      0
Warehouse capacity added (t):   200000
Current warehouse capacity (t): 0


### 3.2.4 Loading station investment decision
In this setup, it is assumed that the loading station has a utilisation rate of 60%. The loading station investment is triggered whenever the yearly loading capacity equals 80% of yearly demand, taking the utilisation rate into account. Once triggered, the loading rate is expanded to accomodate 120% of yearly throughput in steps of 300 t/h

In [10]:
# at t=0 import loading station class from infrastructure package and run initial laoding station configuration
if parameters.timestep == 0:
    stations = infra.stations
    stations = invest.initial_station_setup(stations)
    
# for each time step, check whether pending assets come online
stations = invest.station_online_transition(stations)
    
# for each time step, decide whether to invest in storage
stations_invest_decision = invest.station_invest_decision(stations)

# if investments are needed, calculate how much extra capacity should be added
if stations_invest_decision == 'Invest in loading stations':
    stations = invest.station_expansion(stations)
else:
    stations[0].delta = 0

print ('Loading station capacity added (t/h):  ', stations[0].delta)
print ('Current loading station capacity (t/h):', stations[0].online_capacity)

Loading station capacity added (t/h):   300
Current loading station capacity (t/h): 0


### 3.2.5 Conveyor investment decision
#### 3.2.5.1 Quay conveyor
In this setup, the quay-side conveyor investment dicision is triggered whenever the the crane investment is triggered. The conveyor capacity is always sufficient to cope with the cranes' peak unloading capacity. It is assumed that each additional conveyor built increases conveying capacity by 400 t/h.

In [11]:
# at t=0 import conveyor class from infrastructure package and run initial conveyor configuration
if parameters.timestep == 0:
    q_conveyors = infra.q_conveyors
    q_conveyors = invest.initial_conveyor_setup(q_conveyors)
    
# for each time step, check whether pending assets come online
q_conveyors = invest.conveyor_online_transition(q_conveyors)
    
# for each time step, decide whether to invest in quay conveyors
invest_decision = invest.quay_conveyor_invest_decision(q_conveyors, cranes)
quay_conveyor_invest_decision = invest_decision[0]
q_conveyors = invest_decision[1]

# if investments are needed, calculate how much extra capacity should be added
if quay_conveyor_invest_decision == 'Invest in quay conveyors':
    q_conveyors = invest.quay_conveyor_expansion(q_conveyors, cranes)
else:
    q_conveyors[0].delta = 0

print ('Quay conveyor length (m):             ', q_conveyors[0].length)
print ('Quay conveying capacity added (t/h):  ', q_conveyors[0].delta)
print ('Current quay conveying capacity (t/h):', q_conveyors[0].online_capacity)

Quay conveyor length (m):              500
Quay conveying capacity added (t/h):   2800
Current quay conveying capacity (t/h): 0


#### 3.2.5.1 Hinterland conveyor
In this setup, the hinterland conveyor investment dicision is triggered whenever the loading station investment is triggered. The conveyor capacity is always sufficient to cope with the hinterland loading stations' capacity. It is assumed that each additional conveyor built increases conveying capacity by 400 t/h.

In [12]:
# at t=0 import conveyor class from infrastructure package and run initial conveyor configuration
if parameters.timestep == 0:
    h_conveyors = infra.h_conveyors
    h_conveyors = invest.initial_conveyor_setup(h_conveyors)
    
# for each time step, check whether pending assets come online
h_conveyors = invest.conveyor_online_transition(h_conveyors)
    
# for each time step, decide whether to invest in hinterland conveyors
invest_decision = invest.hinterland_conveyor_invest_decision(h_conveyors, stations)
hinterland_conveyor_invest_decision = invest_decision[0]
h_conveyors = invest_decision[1]

# if investments are needed, calculate how much extra capacity should be added
if hinterland_conveyor_invest_decision == 'Invest in hinterland conveyors':
    h_conveyors = invest.hinterland_conveyor_expansion(h_conveyors, stations)
else:
    h_conveyors[0].delta = 0

print ('Hinterland conveyor length (m):             ', h_conveyors[0].length)
print ('Hinterland conveying capacity added (t/h):  ', h_conveyors[0].delta)
print ('Current hinterland conveying capacity (t/h):', h_conveyors[0].online_capacity)

Hinterland conveyor length (m):              500
Hinterland conveying capacity added (t/h):   400
Current hinterland conveying capacity (t/h): 0


## 3.3 Business Logic

### 3.3.1 Revenue 

In [13]:
assets = [quays, cranes, storage, stations, q_conveyors, h_conveyors]

In [14]:
# at t=0 import revenue class from infrastructure package
if parameters.timestep == 0:
    revenues = financial.business_logic_objects()[0]
    
# for each time step, compute the incoming revenues
revenues[parameters.timestep].calc()

print ('Maize revenue ($):  ', revenues[timestep].maize)
print ('Soybean revenue ($):', revenues[timestep].soybean)
print ('Wheat revenue ($):  ', revenues[timestep].wheat)
print ('Total revenue ($):  ', revenues[timestep].total)

Maize revenue ($):   10000000
Soybean revenue ($): 0
Wheat revenue ($):   0
Total revenue ($):   10000000


### 3.3.2 Capex

In [15]:
# at t=0 import capex class from infrastructure package
if parameters.timestep == 0:
    capex = financial.business_logic_objects()[1]
    
# for each time step, compute the capex
capex[parameters.timestep].calc(assets)

print ('Quay capex($)           ', capex[timestep].quay)
print ('Crane capex($)          ', capex[timestep].cranes)
print ('Storage capex($)        ', capex[timestep].storage)
print ('Conveyor capex($)       ', capex[timestep].conveyors)
print ('Loading station capex($)', capex[timestep].loading_stations)
print ('Total capex ($):        ', capex[timestep].total)

Quay capex($)            15310720
Crane capex($)           7647500
Storage capex($)         0
Conveyor capex($)        9660000
Loading station capex($) 1300000
Total capex ($):         33918220


### 3.3.3 Labour costs

In [16]:
# at t=0 import labour cost class from infrastructure package
if parameters.timestep == 0:
    labour = financial.business_logic_objects()[2]
    
# for each time step, compute the labour costs
labour[parameters.timestep].calc(assets)

print ('International staff costs($):', labour[timestep].international_salary * labour[timestep].international_staff)
print ('Local staff costs($):        ', labour[timestep].local_salary         * labour[timestep].local_staff)
print ('Operational staff costs($):  ', labour[timestep].operational_salary   * labour[timestep].operational_staff)
print ('Total costs($):              ', labour[timestep].total_costs)

International staff costs($): 420000
Local staff costs($):         188500
Operational staff costs($):   0.0
Total costs($):               608500


### 3.3.4 Maintenance costs

In [17]:
# at t=0 import maintenance cost class from infrastructure package
if parameters.timestep == 0:
    maintenance = financial.business_logic_objects()[3]
    
# for each time step, compute the maintenance costs
maintenance[parameters.timestep].calc(assets)

print ('Quay maintenance costs($):    ', maintenance[timestep].quay)
print ('Crane maintenance costs($):   ', maintenance[timestep].cranes)
print ('Storage maintenance costs($): ', maintenance[timestep].storage)
print ('Conveyor maintenance costs($):', maintenance[timestep].conveyors)
print ('Total maintenance costs($):   ', maintenance[timestep].total_costs)

Quay maintenance costs($):     0
Crane maintenance costs($):    0
Storage maintenance costs($):  0
Conveyor maintenance costs($): 0
Total maintenance costs($):    0


### 3.3.5 Energy costs

In [18]:
energy = financial.business_logic_objects()[4]

### 3.3.6 Demurrage costs

In [19]:
demurrage = financial.business_logic_objects()[5]